## ROUGE score

In [ ]:
import pandas as pd
from IPython.display import display

import tkinter as tk
from tkinter import filedialog

root = tk.Tk()
root.withdraw()
file_path = filedialog.askopenfilename(title="Select  CSV file")
df = pd.read_csv(file_path)

display(df.head())
def rouge1_precision(reference,candidate):
    """
    ROUGE-1 Precision: How many words in candidate also appear in reference, divided by candidate's length
    """
    cand_tokens = set(candidate.lower().split())
    ref_tokens = set(reference.lower().split())
    if len(cand_tokens) == 0:
        return 0.0
    overlap = cand_tokens & ref_tokens
    return len(overlap) / len(cand_tokens)


df['rouge1_precision'] = df.apply(
    lambda row: rouge1_precision(str(row['First_attempt']), str(row['last_attempt'])),
    axis=1
)

df.to_csv("ROUGE_Result.csv", index=False)



## BERTScore

In [ ]:
import pandas as pd
from IPython.display import display
import random
from bert_score import score as bert_score
import tkinter as tk
from tkinter import filedialog

root = tk.Tk()
root.withdraw()
file_path = filedialog.askopenfilename(title="Select  CSV file")
df = pd.read_csv(file_path)


candidates = df['First_attempt'].tolist()
references = df['last_attempt'].tolist()
rows = df['row'].tolist()

tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
max_len = 510

def safe_truncate(text, tokenizer, max_len=510):
    tokens = tokenizer.encode(text, add_special_tokens=True)
    if len(tokens) > max_len:
        # Remove special tokens before truncation if present
        tokens = tokens[:max_len]
        return tokenizer.decode(tokens, skip_special_tokens=True)
    else:
        return text
    
def safe_cast_to_str(x):
    if x is None:
        return ""
    try:
        s = str(x)
        if s.lower() == 'nan':
            return ""
        return s
    except Exception:
        return ""

candidates_trunc = [safe_truncate(safe_cast_to_str(t), tokenizer, max_len) for t in candidates]
references_trunc = [safe_truncate(safe_cast_to_str(t), tokenizer, max_len) for t in references]

P= bert_score(candidates_trunc, references_trunc, lang='en', model_type="emilyalsentzer/Bio_ClinicalBERT", num_layers=12)

for i, (p) in enumerate(zip(P)):
    print(f"Row {i}: Precision={p.item():.3f}")

bert_results = pd.DataFrame({
    'row': rows,
    'BERTScore_Precision': [p.item() for p in P],

})

# Save to CSV
bert_results.to_csv("BERTScore_Result.csv", index=False)


